<a href="https://colab.research.google.com/github/parzival82003/Notez-A-book-GPT/blob/main/cgan1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Load dataset from Google Drive
def load_dataset(file_path):
    df = pd.read_csv(file_path)

    # Rename columns for consistency
    df.rename(columns={
        'Gestational Age (Weeks)': 'Gestational Age',
        'Maternal Age (Years)': 'Maternal Age',
        'Spinal Lesion Size (mm)': 'Spinal Lesion Size',
        'Head Measurements': 'Head Circumference',
        'Label': 'Spina Bifida'
    }, inplace=True)

    # Convert "Spina Bifida" column to binary values
    df['Spina Bifida'] = df['Spina Bifida'].map({'Spina Bifida': 1, 'No Spina Bifida': 0})

    # Extract numerical values from "Spinal Lesion Size"
    df['Spinal Lesion Size'] = df['Spinal Lesion Size'].str.extract(r'(\d+)').astype(float).fillna(0)

    # Extract "HC" from "Head Circumference"
    df['Head Circumference'] = df['Head Circumference'].str.extract(r'HC: (\d+)mm').astype(float)

    # Encode categorical values
    label_encoders = {}
    for col in ['Severity', 'Associated Conditions']:
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col].astype(str))
        label_encoders[col] = le

    df.fillna(0, inplace=True)
    return df, label_encoders

# Preprocess dataset
def preprocess_data(df):
    df = df.drop(columns=['ID', 'Textual Report'], errors='ignore')

    scaler = MinMaxScaler()
    features = df.drop(columns=['Spina Bifida'], errors='ignore')
    scaled_features = scaler.fit_transform(features)

    X = scaled_features
    y = df['Spina Bifida'].values.reshape(-1, 1)  # Ensure y is 2D
    return X, y, scaler

# Enable mixed precision for faster training
tf.keras.mixed_precision.set_global_policy('mixed_float16')

# CGAN Components
latent_dim = 50  # Reduced for faster training
num_classes = 2  # Binary classification

def build_cgan_generator(latent_dim, input_dim, num_classes=2):
    noise_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(1,))

    label_embedding = layers.Embedding(num_classes, latent_dim, input_length=1)(label_input)
    label_embedding = layers.Flatten()(label_embedding)

    combined_input = layers.multiply([noise_input, label_embedding])

    x = layers.Dense(128, activation='relu')(combined_input)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(512, activation='relu')(x)
    output = layers.Dense(input_dim, activation='tanh')(x)

    return models.Model([noise_input, label_input], output)

def build_cgan_discriminator(input_dim, num_classes=2):
    feature_input = layers.Input(shape=(input_dim,))
    label_input = layers.Input(shape=(1,))

    label_embedding = layers.Embedding(num_classes, input_dim, input_length=1)(label_input)
    label_embedding = layers.Flatten()(label_embedding)

    combined_input = layers.multiply([feature_input, label_embedding])

    x = layers.Dense(512, activation='relu')(combined_input)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dense(128, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)

    return models.Model([feature_input, label_input], output)

def train_cgan(X, y, epochs=3000, batch_size=32):
    input_dim = X.shape[1]
    discriminator = build_cgan_discriminator(input_dim)
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    generator = build_cgan_generator(latent_dim, input_dim)

    noise_input = layers.Input(shape=(latent_dim,))
    label_input = layers.Input(shape=(1,))
    generated_sample = generator([noise_input, label_input])

    discriminator.trainable = False
    validity = discriminator([generated_sample, label_input])

    cgan = models.Model([noise_input, label_input], validity)
    cgan.compile(loss='binary_crossentropy', optimizer='adam')

    half_batch = batch_size // 2

    for epoch in range(epochs):
        idx = np.random.randint(0, X.shape[0], half_batch)
        real_samples, real_labels = X[idx], y[idx]

        noise = np.random.randn(half_batch, latent_dim)
        fake_labels = np.random.randint(0, 2, half_batch).reshape(-1, 1)
        fake_samples = generator.predict([noise, fake_labels])

        d_loss_real = discriminator.train_on_batch([real_samples, real_labels], np.ones((half_batch, 1)))
        d_loss_fake = discriminator.train_on_batch([fake_samples, fake_labels], np.zeros((half_batch, 1)))
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        noise = np.random.randn(batch_size, latent_dim)
        valid_labels = np.random.randint(0, 2, batch_size).reshape(-1, 1)
        g_loss = cgan.train_on_batch([noise, valid_labels], np.ones((batch_size, 1)))

        if epoch % 500 == 0:
            print(f"Epoch {epoch} | D Loss: {d_loss[0]} | G Loss: {g_loss}")

    return generator, discriminator

def classify_spina_bifida_cgan(sample, label, scaler, discriminator):
    sample = scaler.transform(sample.reshape(1, -1))
    label = np.array([[label]])  # Ensure correct shape
    prediction = discriminator.predict([sample, label])
    return 'Has Spina Bifida' if prediction > 0.5 else 'No Spina Bifida'

# Load and preprocess dataset
file_path = "/content/drive/MyDrive/dataset/fetal_spina_bifida_100.csv"
user_df, label_encoders = load_dataset(file_path)
X, y, scaler = preprocess_data(user_df)

generator, discriminator = train_cgan(X, y)


Mounted at /content/drive


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


/usr/local/lib/python3.11/dist-packages/keras/src/backend/tensorflow/trainer.py:82: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


Epoch 0 | D Loss: 0.6936054229736328 | G Loss: 0.6931471824645996
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━

In [ ]:
# Select a test sample (e.g., first row) for classification
test_sample = X[0]  # First row from the processed dataset
test_label = 1  # 1 for Spina Bifida, 0 otherwise (modify as needed)

# Predict and print classification result
result = classify_spina_bifida_cgan(test_sample, test_label, scaler, discriminator)
print("Prediction:", result)


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 516ms/step
Prediction: No Spina Bifida


In [ ]:
#selects 5 sample
num_samples = 5
test_indices = np.random.choice(X.shape[0], num_samples, replace=False)

print("Predictions for 5 test samples:")
for idx in test_indices:
    test_sample = X[idx]  # Select a sample from the dataset
    test_label = int(y[idx])  # Get the actual label
    result = classify_spina_bifida_cgan(test_sample, test_label, scaler, discriminator)
    print(f"Sample {idx}: {result} (Actual: {'Has Spina Bifida' if test_label == 1 else 'No Spina Bifida'})")


Predictions for 5 test samples:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Sample 59: No Spina Bifida (Actual: No Spina Bifida)


<ipython-input-5-9622af28bd84>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
<ipython-input-5-9622af28bd84>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Sample 69: No Spina Bifida (Actual: No Spina Bifida)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
Sample 36: No Spina Bifida (Actual: Has Spina Bifida)


<ipython-input-5-9622af28bd84>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
<ipython-input-5-9622af28bd84>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Sample 99: No Spina Bifida (Actual: No Spina Bifida)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Sample 32: No Spina Bifida (Actual: Has Spina Bifida)


<ipython-input-5-9622af28bd84>:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [ ]:
# Select 5 test samples: at least 2 with Spina Bifida and 2 without
has_spina_bifida_indices = np.where(y == 1)[0]
no_spina_bifida_indices = np.where(y == 0)[0]

# Ensure we have at least 2 samples from each class
num_samples_per_class = min(2, len(has_spina_bifida_indices), len(no_spina_bifida_indices))

# Randomly select indices
selected_indices = np.concatenate([
    np.random.choice(has_spina_bifida_indices, num_samples_per_class, replace=False),
    np.random.choice(no_spina_bifida_indices, num_samples_per_class, replace=False)
])

# Fill up to 5 samples if possible
remaining_indices = np.setdiff1d(np.arange(len(y)), selected_indices)
if len(remaining_indices) > 0:
    selected_indices = np.concatenate([selected_indices, np.random.choice(remaining_indices, 5 - len(selected_indices), replace=False)])

# Classify and print results
print("\nPredictions for 5 test samples:")
for idx in selected_indices:
    test_sample = X[idx]  # Select a sample from the dataset
    test_label = int(y[idx])  # Get the actual label
    result = classify_spina_bifida_cgan(test_sample, test_label, scaler, discriminator)
    actual_label = "Has Spina Bifida" if test_label == 1 else "No Spina Bifida"
    print(f"Sample {idx}: {result} (Actual: {actual_label})")



Predictions for 5 test samples:


<ipython-input-6-063c2a7b072a>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Sample 16: No Spina Bifida (Actual: Has Spina Bifida)


<ipython-input-6-063c2a7b072a>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Sample 62: No Spina Bifida (Actual: Has Spina Bifida)


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Sample 65: No Spina Bifida (Actual: No Spina Bifida)


<ipython-input-6-063c2a7b072a>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Sample 63: No Spina Bifida (Actual: No Spina Bifida)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step

<ipython-input-6-063c2a7b072a>:23: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  test_label = int(y[idx])  # Get the actual label
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Sample 71: No Spina Bifida (Actual: No Spina Bifida)
